<a href="https://colab.research.google.com/github/Pilo1961/QuestionAnswer_System/blob/master/code/question_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Preguntas y Respuestas
## Métodos Analíticos

In [0]:
import numpy as np
import pandas as pd

In [47]:
#load data from google drive
from google.colab import drive
drive.mount('/content/drive')
path= "/content/drive/My Drive/data/train-v2.0.json"
train=pd.read_json(path,orient='columns')
path= "/content/drive/My Drive/data/dev-v2.0.json"
test=pd.read_json(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
with open(path) as f: 
    d = json.load(f) 
  
# lets put the data into a pandas df 
# clicking on raw_nyc_phil.json under "Input Files" 
# tells us parent node is 'programs' 
test = json_normalize(d['data'])  
test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,title,paragraphs
0,Normans,[{'qas': [{'question': 'In what country is Nor...
1,Computational_complexity_theory,[{'qas': [{'question': 'What branch of theoret...
2,Southern_California,[{'qas': [{'question': 'What is Southern Calif...
3,Sky_(United_Kingdom),[{'qas': [{'question': 'What company was forme...
4,Victoria_(Australia),[{'qas': [{'question': 'What kind of economy d...


In [131]:
nycphil2=pd.json_normalize(data=d['data'],record_path=['paragraphs'],meta=['title'])
nycphil2.head()

,qas,context,title
0,[{'question': 'In what country is Normandy loc...,The Normans (Norman: Nourmands; French: Norman...,Normans
1,[{'question': 'Who was the duke in the battle ...,"The Norman dynasty had a major political, cult...",Normans
2,[{'question': 'What is the original meaning of...,"The English name ""Normans"" comes from the Fren...",Normans
3,[{'question': 'When was the Duchy of Normandy ...,"In the course of the 10th century, the initial...",Normans
4,[{'question': 'Who upon arriving gave the orig...,"Before Rollo's arrival, its populations did no...",Normans


In [153]:
nycphil3=json_normalize(data=d['data'],record_path=['paragraphs','qas'],meta=['title',['paragraph','context']])
nycphil3.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,question,id,answers,is_impossible,plausible_answers,title,paragraph.context
0,In what country is Normandy located?,56ddde6b9a695914005b9628,"[{'text': 'France', 'answer_start': 159}, {'te...",False,NaN,Normans,The Normans (Norman: Nourmands; French: Norman...
1,When were the Normans in Normandy?,56ddde6b9a695914005b9629,"[{'text': '10th and 11th centuries', 'answer_s...",False,NaN,Normans,The Normans (Norman: Nourmands; French: Norman...
2,From which countries did the Norse originate?,56ddde6b9a695914005b962a,"[{'text': 'Denmark, Iceland and Norway', 'answ...",False,NaN,Normans,The Normans (Norman: Nourmands; French: Norman...
3,Who was the Norse leader?,56ddde6b9a695914005b962b,"[{'text': 'Rollo', 'answer_start': 308}, {'tex...",False,NaN,Normans,The Normans (Norman: Nourmands; French: Norman...
4,What century did the Normans first gain their ...,56ddde6b9a695914005b962c,"[{'text': '10th century', 'answer_start': 671}...",False,NaN,Normans,The Normans (Norman: Nourmands; French: Norman...


In [152]:
nycphil3["plausible_answers"]

0                                                      NaN
1                                                      NaN
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
11868                                                  NaN
11869         [{'text': 'pound-force', 'answer_start': 4}]
11870     [{'text': 'kilogram-force', 'answer_start': 82}]
11871    [{'text': 'kilogram-force', 'answer_start': 195}]
11872    [{'text': 'kilogram-force', 'answer_start': 383}]
Name: plausible_answers, Length: 11873, dtype: object

In [162]:
nycphil4=json_normalize(data=d['data'],
                        record_path=['paragraphs','qas','answers'],
                        meta=['title',
                              ['paragraph','context'],
                              ['paragraph','qas','question'],
                              ['paragraph','qas','id'],
                              ['paragraph','qas','is_impossible']])#
                              #['paragraph','qas','plausible_answers']])
nycphil4

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,text,answer_start,title,paragraph.context,paragraph.qas.question,paragraph.qas.id,paragraph.qas.is_impossible
0,France,159,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,56ddde6b9a695914005b9628,False
1,France,159,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,56ddde6b9a695914005b9628,False
2,France,159,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,56ddde6b9a695914005b9628,False
3,France,159,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,56ddde6b9a695914005b9628,False
4,10th and 11th centuries,94,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,56ddde6b9a695914005b9629,False
...,...,...,...,...,...,...,...
20297,sthène,665,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,5737aafd1c456719005744ff,False
20298,sthène,665,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,5737aafd1c456719005744ff,False
20299,sthène,665,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,5737aafd1c456719005744ff,False
20300,sthène,665,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,5737aafd1c456719005744ff,False


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,text,answer_start,title
0,France,159,Normans
1,France,159,Normans
2,France,159,Normans
3,France,159,Normans
4,10th and 11th centuries,94,Normans
...,...,...,...
20297,sthène,665,Force
20298,sthène,665,Force
20299,sthène,665,Force
20300,sthène,665,Force


In [0]:
]